In [ ]:
from datasets import load_dataset
import pandas as pd

# Load the dataset
dataset = load_dataset("neuralwork/arxiver")

# Convert to pandas DataFrame for easier viewing
df = dataset['train'].to_pandas()

# Show the first 5 papers with title and abstract
# print(df['markdown'].iloc[0])

# df[['markdown']].to_csv('raw_paper_data/markdown_only.csv', index=False)
type(df['markdown'].values)

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
separators = ["\n\n", "\n", " ", "", "."]

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(separators=separators, chunk_size=1000, chunk_overlap=5)

# Split your docs into texts
texts_paper = text_splitter.create_documents(df['markdown'].values)

# Embeddings
hg_embeddings = HuggingFaceEmbeddings()

In [ ]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma_rag/'
economic_langchain_chroma = Chroma.from_documents(
    documents=texts_paper,
    collection_name="humanizer_data",
    embedding=hg_embeddings,
    persist_directory=persist_directory
)